In [57]:
import glob
import csv
import pandas as pd
from pathlib import Path
from datetime import *
import math

匯入sensor data

In [62]:
allFiles = glob.glob("../watch"+"*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    list_.append(file_)
list_ = sorted(list_)

#有幾天的資料
csv_num = int(len(list_)/7)

for i in range(csv_num):
    if i==0:
        hR = pd.read_csv(list_[i])
        heading = pd.read_csv(list_[i+csv_num])
        xAcc = pd.read_csv(list_[i+2*csv_num])
        xMag = pd.read_csv(list_[i+3*csv_num])
        yAcc = pd.read_csv(list_[i+4*csv_num])
        yMag = pd.read_csv(list_[i+5*csv_num])
        zMag = pd.read_csv(list_[i+6*csv_num])
    else:
        hR = pd.concat([hR,pd.read_csv(list_[i])], join='inner').reset_index(drop=True)
        heading = pd.concat([heading, pd.read_csv(list_[i+csv_num])], join='inner').reset_index(drop=True)
        xAcc = pd.concat([xAcc,pd.read_csv(list_[i+2*csv_num])], join='inner').reset_index(drop=True)
        xMag = pd.concat([xMag,pd.read_csv(list_[i+3*csv_num])], join='inner').reset_index(drop=True)
        yAcc = pd.concat([yAcc,pd.read_csv(list_[i+4*csv_num])], join='inner').reset_index(drop=True)
        yMag = pd.concat([yMag,pd.read_csv(list_[i+5*csv_num])], join='inner').reset_index(drop=True)
        zMag = pd.concat([zMag,pd.read_csv(list_[i+6*csv_num])], join='inner').reset_index(drop=True)
        
    hR = hR.dropna(how='all')
    heading = heading.dropna(how='all')
    xAcc = xAcc.dropna(how='all')
    xMag = xMag.dropna(how='all')
    yAcc = yAcc.dropna(how='all')
    yMag = yMag.dropna(how='all')
    zMag = zMag.dropna(how='all')
    
    min_length = min(len(hR),len(heading),len(xAcc),len(xMag),len(yAcc),len(yMag), len(zMag))
    hR = hR[:min_length]
    heading = heading[:min_length]
    xAcc = heading[:min_length]
    xMag = xMag[:min_length]
    yAcc = yAcc[:min_length]
    yMag = yMag[:min_length]
    zMag = zMag[:min_length]
print(min_length)

731195


In [67]:
print(hR['datetime'][448916])

2022-10-29 23:31:58


sensor data 與 activity 連結 
data_len筆資料算同一個data

In [104]:
sensors = [hR,heading,xAcc,xMag,yAcc,yMag,zMag]

# dt_obj = datetime.strptime(hR[0]['datetime'][0][2:], '%y-%m-%d %H:%M:%S')
# dt_ = datetime.strptime(hR[0]['datetime'][1][2:], '%y-%m-%d %H:%M:%S')
# print(abs(dt_-dt_obj))


data_len = 400

raw_act = pd.read_csv('../activity.csv')
act_index = 0
dt_act = datetime.strptime(raw_act['datetime'][0][2:], '%y/%m/%d %H:%M')
dt_act_next = datetime.strptime(raw_act['datetime'][1][2:], '%y/%m/%d %H:%M')
cur_act = raw_act['activity'][0]

data_list = []
data_count = 0

values = []
acts = []
val = []

for index in range(448919,min_length):
    data_count = data_count + 1
#     print(index)
#     print(hR['datetime'][int(index)][2:])
    dt = datetime.strptime(hR['datetime'][int(index)][2:], '%y-%m-%d %H:%M:%S')


    val.append(hR['value'][index])
    for s in sensors[1:]:
        val.append(s['value'][index])
        
    if(dt > dt_act_next and act_index < len(raw_act)-2):
        dt_act = dt_act_next
        act_index = act_index + 1
        dt_act_next = datetime.strptime(raw_act['datetime'][act_index+1][2:], '%y/%m/%d %H:%M')

    if(raw_act['activity'][act_index] != cur_act):
        cur_act = raw_act['activity'][act_index] 
        val = []
        data_count = 0

    if(data_count == data_len):
        if(cur_act != 'other' not in val):
            acts.append([cur_act])
            values.append(val)
        val = []
        data_count = 0


In [105]:
print(csv_num)
print(len(values))

45
612


In [106]:
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#shuffle data
temp = list(zip(values, acts))
shuffle(temp)
x, y = zip(*temp)



In [ ]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=0.9)
# x = pca.fit_transform(x)

In [107]:
with open("preprocessed_x.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(x)

with open("preprocessed_y.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(y)


In [103]:
print(len(x))

835
